In [11]:
# ! pip install  --index "http://pypi/simple" --trusted-host pypi  hyperopt

# Debug

In [5]:
import sys, os, gc
import numpy as np
import pandas as pd
import pickle
sys.path.append("./global")
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models.callbacks import CallbackAny2Vec
from helper import ReadCSV, Timer, ExtractFeature, ORI_TRAIN_NAMES, ORI_TRAIN_DTYPE, ORI_TEST_NAMES, ORI_TEST_DTYPE, AnalysisCSV, OFFLINE, cal_sim
import dist_utils, ngram_utils
test_file = "./stage1/input/test.csv"
base_feature_save_dir = "./stage1/output/"
if OFFLINE:
    base_prefix = "debug_"
    train_file = "./stage1/input/train_last_50w.csv"
    CHUNK_SIZE = 500000
else:
    base_prefix = "online_"
    train_file = "./stage1/input/train_last_1000w.csv"
    CHUNK_SIZE = 1000000
print(OFFLINE)

False


In [18]:
AnalysisCSV("./stage1/output/vector_space/online_tfidf_model_ngram3.bin")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [16]:
regen_tfidf_model = False
if OFFLINE:
    tfidf_model_file = feature_save_dir+"debug_tfidf_model.bin"
else:
    tfidf_model_file = feature_save_dir+"online_tfidf_model.bin"

In [10]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
feature_save_dir = base_feature_save_dir + "vector_space/"
train_feature_prefix = base_prefix + "trainVectorSpace"
test_feature_prefix = base_prefix + "testVectorSpace"
! ls -alh ./stage1/output/vector_space/

total 1.3G
drwxr-xr-x 2 kesci root  4.0K Jun 26 15:50 .
drwxr-xr-x 7 kesci root  4.0K Jun 19 09:08 ..
-rw-r--r-- 1 kesci users  89M Jun 20 11:43 debug_tfidf_model.bin
-rw-r--r-- 1 kesci users  12M Jun 26 15:18 debug_trainVectorSpace_tfidfVecSim.csv.gz
-rw-r--r-- 1 kesci users 119M Jun 26 15:52 online_testVectorSpace_tfidfVecSim.csv.gz
-rw-r--r-- 1 kesci users 803M Jun 20 16:59 online_tfidf_model.bin
-rw-r--r-- 1 kesci users 239M Jun 26 15:43 online_trainVectorSpace_tfidfVecSim.csv.gz


In [11]:
from sklearn.metrics.pairwise import paired_distances
from np_utils import try_divide
import math
regen_tfidf_vec_simv1 = True

In [12]:
def csr_jaccard_sim(x, y):
    xy = x.multiply(y).sum()
    x_norm = x.power(2).sum()
    y_norm = y.power(2).sum()
    return try_divide(xy, math.sqrt(x_norm)+math.sqrt(y_norm)-xy)

In [13]:
similar_dis =[
    csr_jaccard_sim
]

In [14]:
def process_tfidf_vec_simv1(df, prefix, savefile):
    with Timer("cal tfidf vec"):
        with open(tfidf_model_file, "rb") as ff:
            tfidf_model = pickle.load(ff)
        q_vec = tfidf_model.transform(df["query"])
        t_vec = tfidf_model.transform(df["title"])
    with Timer("cal tfidf sim"):
        for _ in similar_dis:
            name = _.__name__
            with Timer("cal {} sim".format(name)):
                df["{}_{}".format(prefix, name)] = list(map(_, q_vec, t_vec))
    del tfidf_model, q_vec, t_vec
    return df

In [15]:
# 提取训练数据特征
if regen_tfidf_vec_simv1:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "tfidfVecSimV1", 
        process_func=process_tfidf_vec_simv1, names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, 
        process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], 
        drop_last_cols=['query', 'title'])

----->Started 'extract tfidfVecSimV1 feature' block...
Mem. usage decreased to  7.63 Mb (0.0% reduction)
----->Started 'cal tfidf vec' block...
----->Finished 'cal tfidf vec' block, time used:27.99s.
----->Started 'cal tfidf sim' block...
----->Started 'cal csr_jaccard_sim sim' block...
----->Finished 'cal csr_jaccard_sim sim' block, time used:1465.61s.
----->Finished 'cal tfidf sim' block, time used:1465.7s.
----->Finished 'extract tfidfVecSimV1 feature' block, time used:1494.75s.


KeyboardInterrupt: 

In [4]:
# AnalysisCSV("./stage2/output/feature_selection/features_weight_simple.csv", print_rows=100)

In [24]:
# AnalysisCSV("./stage1/input/train_last_50w.csv", names=ORI_TRAIN_NAMES)
# AnalysisCSV("./stage1/input/train_last_1000w.csv", names=ORI_TRAIN_NAMES)
# AnalysisCSV("./stage1/input/test.csv", names=ORI_TRAIN_NAMES)

In [30]:
# AnalysisCSV("./stage1/output/magic/online_trainMagic_ctrRate.csv.gz")

In [6]:
regen_word2vec_sen_vec_sim = True
from scipy.stats import skew, kurtosis
process_word2vec_sen_vec_mode = "tfidf"
from scipy.spatial.distance import braycurtis, canberra, chebyshev, cityblock, correlation, cosine, euclidean, sqeuclidean
similar_dis = [
    braycurtis, 
    canberra, chebyshev, 
    cityblock, 
    cosine, euclidean, 
    sqeuclidean
]
# similar_radio = [
#     fuzz.ratio,
#     fuzz.partial_ratio,
#     fuzz.token_sort_ratio
# ]
similar_arr = similar_dis
print(similar_arr)


[<function braycurtis at 0x7fd61cb2f510>, <function canberra at 0x7fd61cb2f598>, <function chebyshev at 0x7fd61cb2f488>, <function cityblock at 0x7fd61cb2f378>, <function cosine at 0x7fd61cb2f0d0>, <function euclidean at 0x7fd61cb93ea0>, <function sqeuclidean at 0x7fd61cb93f28>]


In [4]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models.callbacks import CallbackAny2Vec
word2vec_model_file = "./stage1/input/word2vec.kv"
tfidf_model_file = "./stage1/output/vector_space/online_tfidf_model.bin"
import pickle
feature_save_dir = base_feature_save_dir + "word2vec/"
train_feature_prefix = base_prefix + "trainWord2vec"
test_feature_prefix = base_prefix + "testWord2vec"
! ls -alh ./stage1/output/word2vec

total 914M
drwxr-xr-x 2 kesci root  4.0K Jun 24 11:33 .
drwxr-xr-x 7 kesci root  4.0K Jun 19 09:08 ..
-rw-r--r-- 1 kesci users 2.0M Jun 24 11:33 debug_trainWord2vec_lenRel.csv.gz
-rw-r--r-- 1 kesci users  29M Jun 22 01:17 debug_trainWord2vec_senVecavgSim.csv.gz
-rw-r--r-- 1 kesci users  12M Jun 23 14:53 debug_trainWord2vec_senVecavgSK.csv.gz
-rw-r--r-- 1 kesci users 8.8M Jun 23 17:46 debug_trainWord2vec_senVecavgWmd.csv.gz
-rw-r--r-- 1 kesci users 282M Jun 21 12:33 online_testWord2vec_senVecavgSim.csv.gz
-rw-r--r-- 1 kesci users 564M Jun 21 09:21 online_trainWord2vec_senVecavgSim.csv.gz
-rw-r--r-- 1 kesci users  18M Jun 24 09:14 online_trainWord2vec_senVecavgWmdSK.csv.gz


In [7]:
if regen_word2vec_sen_vec_sim:
    word2vec_model = KeyedVectors.load(word2vec_model_file, mmap='r')
    with open(tfidf_model_file, "rb") as ff:
        tfidf_model = pickle.load(ff)
    tfidf_vocab = tfidf_model.vocabulary_

In [40]:
def getTfidfWeights1(raw_docs):
    global tfidf_model
    with Timer("get tfidf weights"):
        # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
        X = tfidf_model.transform(raw_documents=raw_docs)
        row, col = X.nonzero()
        for i, _ in enumerate(raw_docs):
            for __ in _.split():
                tmp_col = col[row==i]
                tmp_row = row[row==i]
                
        print(type(X), X.shape)
        print(X.nonzero())
        print(X.data)
        print(X.indices)
    #     for i, doc in enumerate(raw_docs):
            
    # res = [{_: X[i][tfidf_vocab[_]] for _ in doc.split() if _ in tfidf_vocab} for i, doc in enumerate(raw_docs)]
    # del X
    # gc.collect()
    # return res

In [49]:
print(tfidf_vocab["12"])

697832


In [41]:
raw_docs = ["20 20 21 123 12 22", "22 22"]
tmp = getTfidfWeights1(raw_docs)

----->Started 'get tfidf weights' block...
<class 'scipy.sparse.csr.csr_matrix'> (2, 8848846)
(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1], dtype=int32), array([ 697832,  701020,  798245,  798411, 2692668, 2695257, 2695414,
       2881256, 3040859, 3040859, 3045293], dtype=int32))
[0.14323257 0.42832573 0.21215492 0.52686526 0.30584918 0.48816456
 0.31331434 0.16380954 0.12875169 0.48506472 0.87447826]
[ 697832  701020  798245  798411 2692668 2695257 2695414 2881256 3040859
 3040859 3045293]
----->Finished 'get tfidf weights' block, time used:0.12s.


In [5]:
def sent2vec1(text, X):
    # 计算embedding带权均值
    global word2vec_model, tfidf_vocab
    return np.nan_to_num(
          np.array([np.array(word2vec_model[w]) * X[tfidf_vocab[w]]
          or [0.0] * 200 for w in text.split() if w in word2vec_model] 
          or [0.0] * 200
          ).mean(axis=0)
      )

In [6]:
def process_word2vec_sen_vec(df, prefix, savefile):
    with Timer("cal sent vec"):
        global process_word2vec_sen_vec_mode, tfidf_model
        if process_word2vec_sen_vec_mode == "avg":
            q_sen_vec = list(map(sent2vec, df["query"]))
            t_sen_vec = list(map(sent2vec, df["title"]))
        elif process_word2vec_sen_vec_mode == "tfidf":
            # q_sen_weights = getTfidfWeights(df["query"])
            # t_sen_weights = getTfidfWeights(df["title"])
            with Timer("cal X weights"):
                X = tfidf_model.transform(raw_documents=df["query"])
            q_sen_vec = list(map(sent2vec1, df["query"], X.todense()))
            del X
            gc.collect()
            with Timer("cal Y weights"):
                Y = tfidf_model.transform(raw_documents=df["title"])
            t_sen_vec = list(map(sent2vec1, df["title"], Y.todense()))
            del Y
            gc.collect()
        # savefile += "{}.npz".format(process_word2vec_sen_vec_mode)
        # np.savez(savefile, q_sent_vec, t_sent_vec)
        # print("Sent vec saved to {}".format(savefile))
        for _ in similar_arr:
            name = _.__name__
            with Timer("cal {} sim".format(name)):
                df["{}_{}".format(prefix, name)] = cal_sim(q_sen_vec, t_sen_vec, _)
        del q_sen_vec, t_sen_vec
        gc.collect()
    return df

In [32]:
# 提取训练数据特征
if regen_word2vec_sen_vec_sim:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, 
        "senVec{}Sim".format(process_word2vec_sen_vec_mode), 
        process_func=process_word2vec_sen_vec, names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, 
        process_chunkly=True, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], 
        drop_last_cols=['query', 'title'])
# Finished 'extract senVecavgSim feature' block, time used:4160.86s.

----->Started 'extract senVectfidfSim feature' block...
----->Started 'read chunk' block...
----->Finished 'read chunk' block, time used:1.48s.
----->Started 'process chunk' block...
Mem. usage decreased to  7.63 Mb (0.0% reduction)
----->Started 'cal sent vec' block...
----->Started 'cal X weights' block...
----->Finished 'cal X weights' block, time used:7.67s.
----->Started 'cal Y weights' block...
----->Finished 'cal Y weights' block, time used:32.67s.


In [2]:
feature_save_dir = base_feature_save_dir + "magic/"
train_feature_prefix = base_prefix + "trainMagic"
test_feature_prefix = base_prefix + "testMagic"
! ls -alh ./stage1/output/magic/

total 8.1G
drwxr-xr-x 2 kesci root  4.0K Jun 26 12:11 .
drwxr-xr-x 7 kesci root  4.0K Jun 19 09:08 ..
-rw-r--r-- 1 kesci users 5.8K Jun 22 01:22 debug_trainMagic_ctrRateGlobal.csv.gz
-rw-r--r-- 1 kesci users 122K Jun 22 01:23 debug_trainMagic_ctrRateLocal.csv.gz
-rw-r--r-- 1 kesci users  89M Jun 24 15:16 debug_trainMagic_diffSetSimBigramsTitle.npy
-rw-r--r-- 1 kesci users  20M Jun 24 15:33 debug_trainMagic_diffSetSim.csv.gz
-rw-r--r-- 1 kesci users 113M Jun 24 15:25 debug_trainMagic_diffSetSimTrigramsTitle.npy
-rw-r--r-- 1 kesci users  47M Jun 24 15:10 debug_trainMagic_diffSetSimUnigramsTitle.npy
-rw-r--r-- 1 kesci users  62K Jun 25 15:37 debug_trainMagic_queryFreq.csv.gz
-rw-r--r-- 1 kesci users  95K Jun 21 13:59 online_testMagic_ctrRateGlobal.csv.gz
-rw-r--r-- 1 kesci users 1.2M Jun 21 14:09 online_testMagic_ctrRateLocal.csv.gz
-rw-r--r-- 1 kesci users 931M Jun 22 09:34 online_testMagic_diffSetSimBigramsTitle.npy
-rw-r--r-- 1 kesci users 151M Jun 26 12:17 online_testMagic_diffSetSim.

In [13]:
def run_diff_set_len(df, ngram, prefix, savefile, name):
    with Timer("cal diff set"):
        t_savefile = savefile + name +"Title.npy"
        t_savefile = t_savefile.replace("Len", "Sim")
        q_list = df['query'].apply(ngram)
        print(t_savefile)
        if os.path.exists(t_savefile):
            t_list = np.load(t_savefile, allow_pickle=True)
        else:
            pass
            # t_list = GetDiffSet(df["query_id"], df["title"], ngram)
            # np.save(t_savefile, t_list) # 存储处理后的title列表
            # print("File saved to {}".format(t_savefile))
    ## 长度特征
    with Timer("extract length rel"):
        q_len = np.vectorize(len)(q_list)
        t_len = np.vectorize(len)(t_list)
        df['%s_difft-len' % prefix] = t_len
        df['%s_diffqt-len-radio'%prefix] = list(map(lambda a, b: a/b, q_len, t_len))
        df['%s_difftq-len-radio'%prefix] = list(map(lambda a, b: b/a, q_len, t_len))
        df['%s_diffqt-diff'%prefix] = list(map(lambda a, b: abs(a-b), q_len, t_len))
        df['%s_diffqt-max'%prefix] = list(map(lambda a, b: max(a, b), q_len, t_len))
        df['%s_diffqt-min'%prefix] = list(map(lambda a, b: min(a, b), q_len, t_len))
        df['%s_diffqt-avg'%prefix] = list(map(lambda a, b: (a+b)/2, q_len, t_len))
    del q_list, t_list, q_len
    gc.collect()
    return df
    
def process_diff_set_len(df, prefix, savefile):
    # df = run_diff_set_sim(df, ngram_utils.unichars, '%s_%s' % (prefix, 'Unichars'), savefile, "Unichars")
    df = run_diff_set_len(df, ngram_utils.unigrams, '%s_%s' % (prefix, 'unigrams'), savefile, "Unigrams")
    # df = run_diff_set_len(df, ngram_utils.bigrams, '%s_%s' % (prefix, 'bigrams'), savefile, "Bigrams")
    # df = run_diff_set_len(df, ngram_utils.trigrams, '%s_%s' % (prefix, 'trigrams'), savefile, "Trigrams")
    return df

In [14]:
ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "diffSetLen", 
    process_func=process_diff_set_len, 
    names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
    drop_first_cols=['label', 'query_title_id'], drop_last_cols=['query_id', 'query', 'title'])

----->Started 'extract diffSetLen feature' block...
Mem. usage decreased to  9.54 Mb (0.0% reduction)
----->Started 'cal diff set' block...
./stage1/output/magic/debug_trainMagic_diffSetSimUnigramsTitle.npy
----->Finished 'cal diff set' block, time used:2.88s.
----->Started 'extract length rel' block...


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in long_scalars


----->Finished 'extract length rel' block, time used:3.24s.
saved diffSetLen feature to ./stage1/output/magic/debug_trainMagic_diffSetLen.csv.gz
----->Finished 'extract diffSetLen feature' block, time used:17.22s.


### 训练词tfidf

In [5]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
feature_save_dir = base_feature_save_dir + "vector_space/"
train_feature_prefix = base_prefix + "trainVectorSpace"
test_feature_prefix = base_prefix + "testVectorSpace"
tfidf_model_file = feature_save_dir+"online_tfidf_model_ngram3_char.bin"
print(tfidf_model_file)
! ls -alh ./stage1/output/vector_space/
regen_tfidf_model = True

./stage1/output/vector_space/online_tfidf_model_ngram3_char.bin
total 3.8G
drwxr-xr-x 2 kesci root  4.0K Jun 29 06:49 .
drwxr-xr-x 7 kesci root  4.0K Jun 19 09:08 ..
-rw-r--r-- 1 kesci users  89M Jun 20 11:43 debug_tfidf_model.bin
-rw-r--r-- 1 kesci users 1.2M Jun 28 14:44 debug_trainVectorSpace_ngramSimHash.csv.gz
-rw-r--r-- 1 kesci users 374K Jun 29 02:52 debug_trainVectorSpace_ngramSimHashV1.csv.gz
-rw-r--r-- 1 kesci users  25M Jun 27 14:25 debug_trainVectorSpace_tfidfVecLen.csv.gz
-rw-r--r-- 1 kesci users  18M Jun 28 12:50 debug_trainVectorSpace_tfidfVecLenMore.csv.gz
-rw-r--r-- 1 kesci users 4.1M Jun 28 12:59 debug_trainVectorSpace_tfidfVecLenMulti.csv.gz
-rw-r--r-- 1 kesci users  30M Jun 29 06:37 debug_trainVectorSpace_tfidfVecLenV1.csv.gz
-rw-r--r-- 1 kesci users  12M Jun 26 15:18 debug_trainVectorSpace_tfidfVecSim.csv.gz
-rw-r--r-- 1 kesci users 249M Jun 27 14:59 online_testVectorSpace_tfidfVecLen.csv.gz
-rw-r--r-- 1 kesci users 119M Jun 26 15:52 online_testVectorSpace_tfidfVec

In [6]:
def train_tfidf(df, prefix, savefile):
    word_tfidf = TfidfVectorizer(norm="l2",
                                    strip_accents="unicode",
                                    analyzer="char",
                                    ngram_range=(1, 3),
                                    use_idf=True,
                                    smooth_idf=True,
                                    sublinear_tf=True, min_df=5, max_df=0.9)
    new_query = df["query"].unique()
    new_title = df["title"].values
    # print(new_query.shape)
    # print(new_title.shape)
    corpus = np.concatenate([new_query, new_title])
    # print(corpus[:10])
    # print(corpus[10:])
    del df
    gc.collect()
    with Timer("tf-idf fit"):
        word_tfidf.fit(corpus)
        with open(tfidf_model_file, "wb") as f:
            pickle.dump(word_tfidf, f)
            print("Model dumped to {}".format(tfidf_model_file))
    return None

In [4]:
# 提取训练数据特征
if regen_tfidf_model:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "tfidf", 
        process_func=train_tfidf, names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, 
        process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], 
        drop_last_cols=['query', 'title'])

----->Started 'extract tfidf feature' block...
Mem. usage decreased to 152.59 Mb (0.0% reduction)
----->Started 'tf-idf fit' block...


### LCS

In [2]:
from fuzzywuzzy import fuzz
feature_save_dir = base_feature_save_dir + "text_mining_feature/"
train_feature_prefix = base_prefix + "trainTextMining"
test_feature_prefix = base_prefix + "testTextMining"
! ls -alh ./stage1/output/text_mining_feature/

total 1.9G
drwxr-xr-x 2 kesci root  4.0K Jun 26 13:29 .
drwxr-xr-x 7 kesci root  4.0K Jun 19 09:08 ..
-rw-r--r-- 1 kesci users 8.6M Jun 26 13:27 debug_trainTextMining_editSim.csv.gz
-rw-r--r-- 1 kesci users 4.6M Jun 26 13:24 debug_trainTextMining_len.csv.gz
-rw-r--r-- 1 kesci users 2.7M Jun 26 13:24 debug_trainTextMining_lenRel.csv.gz
-rw-r--r-- 1 kesci users  21M Jun 24 12:15 debug_trainTextMining_ngramCharSim.csv.gz
-rw-r--r-- 1 kesci users  34M Jun 25 14:00 debug_trainTextMining_ngramSim.csv.gz
-rw-r--r-- 1 kesci users  20M Jun 26 13:30 debug_trainTextMining_ngramSimV1.csv.gz
-rw-r--r-- 1 kesci users  86M Jun 25 17:42 online_testTextMining_editSim.csv.gz
-rw-r--r-- 1 kesci users  47M Jun 20 13:22 online_testTextMining_len.csv.gz
-rw-r--r-- 1 kesci users  27M Jun 25 16:09 online_testTextMining_lenRel.csv.gz
-rw-r--r-- 1 kesci users 262M Jun 20 16:46 online_testTextMining_ngramSim.csv.gz
-rw-r--r-- 1 kesci users 197M Jun 26 09:13 online_testTextMining_ngramSimV1.csv.gz
-rw-r--r-- 1 ke

In [3]:
import numpy as np
regen_lcs = True
def lcs(seq1, seq2):
    dp = np.zeros((len(seq1)+1, len(seq2)+1))
    for i in range(len(seq1)):
        for j in range(len(seq2)):
            dp[i + 1][j + 1] = max(dp[i + 1][j], dp[i][j], dp[i][j + 1])
            if seq1[i] == seq2[j]:
                dp[i + 1][j + 1] = max(dp[i][j] + 1, dp[i + 1][j + 1])
    seq1_mask = [ 0 for wd in seq1]
    seq2_mask = [ 0 for wd in seq2]
    ii, jj = len(seq1), len(seq2)
    while ii != 0 and jj != 0:
        if dp[ii][jj] == dp[ii - 1][jj - 1] + 1 and seq1[ii - 1] == seq2[jj - 1]:
            seq1_mask[ii - 1] = 1
            seq2_mask[jj - 1] = 1
            ii = ii - 1
            jj = jj - 1
            continue
        if dp[ii][jj] == dp[ii - 1][jj]:
            ii = ii - 1
        elif dp[ii][jj] == dp[ii][jj - 1]:
            jj = jj - 1
        elif dp[ii][jj] == dp[ii - 1][jj - 1]:
            ii = ii - 1
            jj = jj - 1
    seq1_left = [ wd for wd, mk in zip(seq1, seq1_mask) if mk == 0]
    seq2_left = [ wd for wd, mk in zip(seq2, seq2_mask) if mk == 0]
    
    return np.max(dp), ' '.join(seq1_left), ' '.join(seq2_left)

In [4]:
def run_ngram_lcs(df, ngram, prefix):
    with Timer("cal ngram lcs"):
        q_list = df['query'].apply(ngram)
        t_list = df['title'].apply(ngram)
        lcs_v = np.array(list(map(lambda x,y: lcs(x, y), q_list, t_list)))
        df["{}Lcs".format(prefix)] = lcs_v[:, 0]
        del q_list, t_list
        gc.collect()
    return df

def process_ngram_lcs(df, prefix, savefile):
    # df = run_ngram_simhash(df, ngram_utils.unichars, '%s_%s' % (prefix, 'unichars'))
    # df = run_ngram_simhash(df, ngram_utils.bichars, '%s_%s' % (prefix, 'bichars'))
    # df = run_ngram_simhash(df, ngram_utils.trichars, '%s_%s' % (prefix, 'trichars'))
    df = run_ngram_lcs(df, ngram_utils.unigrams, '%s_%s' % (prefix, 'unigrams'))
    # df = run_ngram_lcs(df, ngram_utils.bigrams, '%s_%s' % (prefix, 'bigrams'))
    # df = run_ngram_lcs(df, ngram_utils.trigrams, '%s_%s' % (prefix, 'trigrams'))
    return df 

In [3]:
# 提取训练数据特征
if regen_lcs:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "ngramLcs", 
        process_func=process_ngram_lcs, 
        names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], drop_last_cols=['query', 'title'])

----->Started 'extract ngramLcs feature' block...
Mem. usage decreased to 152.59 Mb (0.0% reduction)
----->Started 'cal ngram lcs' block...


## 目录查看

In [5]:
!ls -alh stage1/input
!ls -alh stage1/output/text_mining_feature
!ls -alh stage1/output/vector_space
!ls -alh stage1/output/word2vec
!ls -alh stage1/output/magic
# PrintCSV(train_file, names=ORI_TRAIN_NAMES)
# !cd stage1/input; mv word2vec_sentences.txt word2vec_sentences_1000w.txt
# !cd stage1/input; rm word2vec_sentences_1000w.txt
# ! rm stage1/output/vector_space/online_trainVectorSpace_lsi_feature.npz

total 3.0G
drwxr-xr-x 2 kesci root  4.0K Jun 18 14:52 .
drwxr-xr-x 6 kesci users 4.0K Jun  9 06:09 ..
-rw-r--r-- 1 kesci users 426M Jun 14 13:13 test.csv
-rw-r--r-- 1 kesci users 880M Jun 14 13:11 train_last_1000w.csv
-rw-r--r-- 1 kesci users  44M Jun 15 12:48 train_last_50w.csv
-rw-r--r-- 1 kesci users  41M Jun 18 14:52 word2vec.kv
-rw-r--r-- 1 kesci users 546M Jun 18 14:52 word2vec.kv.vectors.npy
-rw-r--r-- 1 kesci users  44M Jun 15 06:06 word2vec.model
-rw-r--r-- 1 kesci users 546M Jun 15 06:06 word2vec.model.trainables.syn1neg.npy
-rw-r--r-- 1 kesci users 546M Jun 15 06:06 word2vec.model.wv.vectors.npy
total 1.9G
drwxr-xr-x 2 kesci root  4.0K Jun 26 11:28 .
drwxr-xr-x 7 kesci root  4.0K Jun 19 09:08 ..
-rw-r--r-- 1 kesci users 8.6M Jun 25 13:50 debug_trainTextMining_editSim.csv.gz
-rw-r--r-- 1 kesci users 4.6M Jun 25 12:26 debug_trainTextMining_len.csv.gz
-rw-r--r-- 1 kesci users 2.0M Jun 25 12:29 debug_trainTextMining_lenRel.csv.gz
-rw-r--r-- 1 kesci users  21M Jun 24 12:15 debug_

In [3]:
!ls -alh stage2/input
!ls -alh stage2/output

total 2.1G
drwxr-xr-x 2 kesci root  4.0K Jun 17 15:38 .
drwxr-xr-x 6 kesci users 4.0K Jun  9 13:28 ..
-rw-r--r-- 1 kesci users 2.2K Jun 17 07:11 debug_feature_names.csv
-rw-r--r-- 1 kesci users 134M Jun 17 07:11 debug_train_concat_feature.npy
-rw-r--r-- 1 kesci users 7.7M Jun 16 06:08 debug_train_for_cal_auc.npy
-rw-r--r-- 1 kesci users 3.9M Jun 15 15:30 debug_train_labels.npy
-rw-r--r-- 1 kesci users 1.4K Jun 17 15:39 online_feature_names.csv
-rw-r--r-- 1 kesci users 611M Jun 17 15:39 online_test_concat_feature.npy
-rw-r--r-- 1 kesci users 1.2G Jun 17 15:38 online_train_concat_feature.npy
-rw-r--r-- 1 kesci users  77M Jun 15 15:13 online_train_labels.npy
total 52K
drwxr-xr-x 4 kesci root  4.0K Jun 17 16:54 .
drwxr-xr-x 6 kesci users 4.0K Jun  9 13:28 ..
-rw-r--r-- 1 kesci users 1.3K Jun 16 16:44 lgb_gbdt_debug_len_ngramSimilarFuzzy_lsiDis_lsiFeat_0.5593_1560703452.json
-rw-r--r-- 1 kesci users 1.4K Jun 17 07:31 lgb_gbdt_debug_len_ngramSimilarFuzzy_lsiDis_lsiFeat_0.5593_1560756714.json

In [9]:
# !ls -alh stage2/output/train
!ls -alh stage2/output/test

total 2.3G
drwxr-xr-x 2 kesci root  12K Jun 29 23:24 .
drwxr-xr-x 5 kesci root  12K Jun 29 04:09 ..
-rw-r--r-- 1 kesci users 39M Jun 17 16:19 lgb_gbdt_1560788289_auc0.569244_loss0.562268_std0.000796.npy
-rw-r--r-- 1 kesci users 39M Jun 17 16:27 lgb_gbdt_1560788741_auc0.567697_loss0.562857_std0.000960.npy
-rw-r--r-- 1 kesci users 39M Jun 17 16:35 lgb_gbdt_1560789238_auc0.568515_loss0.562582_std0.000935.npy
-rw-r--r-- 1 kesci users 39M Jun 17 16:44 lgb_gbdt_1560789779_auc0.568970_loss0.562726_std0.001191.npy
-rw-r--r-- 1 kesci users 39M Jun 17 16:54 lgb_gbdt_1560790350_auc0.569695_loss0.562091_std0.000796.npy
-rw-r--r-- 1 kesci users 39M Jun 18 14:41 lgb_gbdt_1560868795_auc0.570428_loss0.562419_std0.001150.npy
-rw-r--r-- 1 kesci users 39M Jun 18 14:56 lgb_gbdt_1560869614_auc0.572052_loss0.561928_std0.001085.npy
-rw-r--r-- 1 kesci users 39M Jun 18 15:09 lgb_gbdt_1560870403_auc0.572468_loss0.561707_std0.000985.npy
-rw-r--r-- 1 kesci users 39M Jun 18 15:33 lgb_gbdt_1560871745_auc0.572409_lo

In [41]:
!cat stage2/output/lgb_gbdt_debug_len_ngram_similarity_0.5633_1560669808.json

{
 "state": 2,
 "tid": 1,
 "spec": null,
 "result": {
  "loss": 0.5632760167257869,
  "auc": 0.5564878203367812,
  "loss_variance": 0.0007281675021687126,
  "status": "ok",
  "file_name": "lgb_gbdt_1560668860_auc0.556500_loss0.563300_std0.000700.npy",
  "q_auc": 0.5204636949875794
 },
 "misc": {
  "tid": 1,
  "cmd": [
   "domain_attachment",
   "FMinIter_Domain"
  ],
  "workdir": null,
  "idxs": {
   "bagging_fraction": [
    1
   ],
   "bagging_freq": [
    1
   ],
   "feature_fraction": [
    1
   ],
   "learning_rate": [
    1
   ],
   "max_bin": [
    1
   ],
   "min_data_in_leaf": [
    1
   ],
   "num_iterations": [
    1
   ],
   "num_leaves": [
    1
   ]
  },
  "vals": {
   "bagging_fraction": [
    0.9
   ],
   "bagging_freq": [
    2.0
   ],
   "feature_fraction": [
    0.9
   ],
   "learning_rate": [
    0.14
   ],
   "max_bin": [
    2
   ],
   "min_data_in_leaf": [
    20.0
   ],
   "num_iterations": [
    80.0
   ],
   "num_leaves": [
    10.0
   ]
  }
 },
 "exp_key": nu

## 结果提交

In [19]:
res_id = "lgb_gbdt_1561850051_auc0.682662_loss0.528813_std0.003102"
out_res_file = "./stage2/output/test/"+res_id+".npy"
test_file = "./stage1/input/test.csv"

In [20]:
out_test_res = np.load(out_res_file)
test_csv = ReadCSV(test_file, names=ORI_TEST_NAMES, dtype=None, iterator=False)
res = pd.DataFrame({
    "query_id": test_csv["query_id"],
    "query_title_id": test_csv["query_title_id"],
    "label":out_test_res
})
res.to_csv("./global/submits/"+res_id+".csv", index=None, header=None)

In [14]:
! ls -lsh ./global/submits

total 4.7G
7.5M -rwxr-xr-x 1 kesci users 7.5M May 25 15:15 kesci_submit
136M -rw-r--r-- 1 kesci users 136M Jun  8 12:26 lgb_gbdt_1559990841_0.547674_0.003729.csv
136M -rw-r--r-- 1 kesci users 136M Jun  9 00:27 lgb_gbdt_1560015399_0.548471_0.003714.csv
136M -rw-r--r-- 1 kesci users 136M Jun  9 14:20 lgb_gbdt_1560088510_0.559713_0.559681_0.559278_0.000611.csv
136M -rw-r--r-- 1 kesci users 136M Jun  9 23:59 lgb_gbdt_1560101261_0.558418_0.558846_0.561068_0.001116.csv
136M -rw-r--r-- 1 kesci users 136M Jun 10 07:23 lgb_gbdt_1560145986_0.559661_0.559640_0.559232_0.000524.csv
136M -rw-r--r-- 1 kesci users 136M Jun 12 13:11 lgb_gbdt_1560343909_0.566367_0.558761_0.001295.csv
136M -rw-r--r-- 1 kesci users 136M Jun 14 12:13 lgb_gbdt_1560513856_0.566747_0.561561_0.000980.csv
137M -rw-r--r-- 1 kesci users 137M Jun 16 03:13 lgb_gbdt_1560654538_0.553203_0.564397_0.000363.csv
136M -rw-r--r-- 1 kesci users 136M Jun 16 05:43 lgb_gbdt_1560662390_0.557880_0.563663_0.000493.csv
136M -rw-r--r-- 1 kesci user

In [21]:
!cd ./global/submits; ./kesci_submit -token 490475a1ae106f67 -file lgb_gbdt_1561850051_auc0.682662_loss0.528813_std0.003102.csv

Kesci Submit Tool
Result File: lgb_gbdt_1561850051_auc0.682662_loss0.528813_std0.003102.csv (135.87 MiB)
Uploaded.       
Submit Success.
{"Stage":0,"Status":0,"ShownInHistory":true,"IsAucResult":true,"Selected":false,"_id":"5d182e99d5eca3002cc2e2b0","Competition":"5cc51043f71088002c5b8840","Team":"5cda7d99b4de58002b94c3f8","UploadDate":"2019-06-30T03:38:01.578Z","Final":true,"Response":"","SubmissionResults":[],"IP":"52.83.252.251","FingerPrint":"","UserAgent":"Go-http-client/1.1","ResultFileName":"156186586329112f2e6.csv","ResultFileRealName":"lgb_gbdt_1561850051_auc0.682662_loss0.528813_std0.003102.csv","ResultFileSize":0,"ReviewInfos":[],"__v":0}



## 进程管理

In [16]:
! ps aux
! kill -9 7926

USER        PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
kesci         1  0.0  0.0 355800 60928 ?        Ssl  Jun29   0:48 /opt/conda/bin
kesci         7  0.0  0.0 186880  5908 ?        Sl   Jun29   0:00 gotty --port 8
kesci       111  0.0  0.0      0     0 ?        Z    Jun29   0:00 [python] <defu
kesci       266  0.0  0.0      0     0 ?        Z    Jun29   0:00 [python] <defu
kesci       405  0.0  0.0      0     0 ?        Z    Jun29   0:00 [python] <defu
kesci       546  0.0  0.0      0     0 ?        Z    Jun29   0:00 [python] <defu
kesci       729  0.0  0.0      0     0 ?        Z    Jun29   0:00 [python] <defu
kesci       959  0.0  0.0      0     0 ?        Z    Jun29   0:00 [python] <defu
kesci      1100  0.0  0.0      0     0 ?        Z    Jun29   0:00 [python] <defu
kesci      1305  0.0  0.0      0     0 ?        Z    Jun29   0:00 [python] <defu
kesci      1385  0.0  0.0      0     0 ?        Z    Jun29   0:00 [python] <defu
kesci      1489  0.0  0.0      0   

## 数据准备

1. 取后1000w条数据用以训练
2. 将test.csv移动到stage1/input文件夹

In [1]:
!tail -n 15000000 "./stage1/input/train_last_1000w.csv" > "./stage1/input/train_last_50w.csv"

In [17]:
!tail -n 5000000 "/home/kesci/input/bytedance/first-round/train.csv" > "./stage1/input/train_last_500w.csv"

In [9]:
# ! cp "/home/kesci/input/bytedance/first-round/test.csv" "./stage1/input/test.csv"